# Chapter 5: Eager and Lazy APIs

In [ ]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.13.0

## Eager API: DataFrame

In [ ]:
import polars as pl

## Lazy API: LazyFrame

In [ ]:
# This raises a SchemaError:
# names_lf = pl.LazyFrame({"name": ["Alice", "Bob", "Charlie"], "age": [25, 30, 35]})

# erroneous_query = names_lf.with_columns(
#     sliced_age=pl.col("age").str.slice(1, 3)
# )

# result_df = erroneous_query.collect()

## Performance Differences

In [ ]:
lf = pl.LazyFrame({"col1": [1, 2, 3], "col2": [4, 5, 6]})

# Some heavy computation
print(lf.collect())

# Recalculates the LazyFrame
print(lf.with_columns(pl.col("col1") + 1).collect())

## Functionality Differences

### Aggregations

### Attributes

### Computation

### Descriptive

### GroupBy

### Exporting

### Manipulation and Selection

### Miscellaneous

## Tips and Tricks

### Going from LazyFrame to DataFrame and Vice Versa

### Joining a DataFrame and a LazyFrame

In [ ]:
# This causes a TypeError
# big_sales_data = pl.LazyFrame(
#     {"sale_id": [101, 102, 103], "amount": [250, 150, 300]}
# )
# sales_metadata = pl.DataFrame(
#     {"sale_id": [101, 102, 103], "category": ["A", "B", "A"]}
# )

# big_sales_data.join(sales_metadata, on="sale_id").collect()

In [ ]:
big_sales_data = pl.LazyFrame(
    {"sale_id": [101, 102, 103], "amount": [250, 150, 300]}
)
sales_metadata = pl.DataFrame(
    {"sale_id": [101, 102, 103], "category": ["A", "B", "A"]}
)

big_sales_data.join(sales_metadata.lazy(), on="sale_id").collect()

### Caching Intermittent Stages

In [ ]:
lf = pl.LazyFrame({"col1": [1, 2, 3], "col2": [4, 5, 6]})

# Some heavy computation
lf = lf.collect().lazy()
print(lf.collect())

# Utilizes the cached LazyFrame
print(lf.with_columns(pl.col("col1") + 1).collect())

## Takeaways